Bibliotecas

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns # pip install seaborn
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler # pip install imbalanced-learn
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Lectura del conjunto de datos

In [ ]:
df = pd.read_csv("mort_hospital.csv")

**Problema**: Alto número de columnas con datos no informados

In [ ]:
df.isnull().any().sum()

**Problema**: Presencia de variables no numéricas

In [ ]:
object_columns = df.select_dtypes(include=['object']).columns
cols = df.select_dtypes([np.number]).columns
print("Columnas con datos categóricos:", len(object_columns))
print("Columnas con datos numéricos:", len(cols))

**Problema**: Las clases de la variable de interés están muy desequilibradas

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='hospital_death',data=df)

Solución al problema de los datos no informados:
- Variables categóricas: rellenar con la moda
- Variables numéricas: rellenar con el promedio

In [ ]:
for i in object_columns:
    df[i].fillna(df[i].mode()[0], inplace=True)

df[cols] = df[cols].fillna(df[cols].mean())

Solución al problema de las variables no numéricas:
- Codificación de variables categóricas

In [ ]:
print("Valores de la variable 'etnicity' antes de la codificación:", df['ethnicity'].unique())
le = LabelEncoder()
df[object_columns] = df[object_columns].apply(le.fit_transform)
print("Valores de la variable 'etnicity' después de la codificación:", df['ethnicity'].unique())

Solución al problema del desequilibrio de la variable de interés:
- Muestreo de la clase más numerosa

In [ ]:
X = df.drop(columns='hospital_death', axis=1)
y = df['hospital_death']

undersample = RandomUnderSampler(sampling_strategy='majority')
X, y = undersample.fit_resample(X, y)

print("Recuento de categorías de salida:", Counter(y))

Selección de características

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

num_k = 10
# El parámetro k determina cuántas características se seleccionan
# El parámetro input_dim de la primera capa de la red deberá ser num_k
kmejores = SelectKBest(mutual_info_classif, k=num_k)
kmejores.fit(X_train, y_train)
X_train.columns[kmejores.get_support()]
X_train_new = kmejores.transform(X_train)
X_test_new =  kmejores.transform(X_test)

Normalización de datos

In [ ]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train_new)
X_test_std = scaler.transform(X_test_new)

### Solución de la tarea 2
**Objetivos**
- Entender el código proporcionado: tratamiento de datos no informados, codificación de variables categóricas, equilibrado de categorías de la variable de interés, selección de características y normalización.
- Diseñar y entrenar una red neuronal para predecir la variable 'hospital_death':
    - Probar con distintos valores de 'num_k' para cambiar el número de características seleccionadas.
    - Diseñar un modelo secuencial ('Sequential()'), cuya primera capa debe especificar como 'input_dim' el valor 'num_k', probando distintos números de capas, funciones de activación (RELU y variantes para las capas ocultas, sigmoide para la capa final), número de neuronas en cada capa, capas 'Dropout', etc.
    - Compilar el modelo estableciendo una función de error ('loss'), el optimizador (puedes cambiar el parámetro 'learning_rate') y las métricas de rendimiento que se calcularán.
    - Ajustar el modelo con los datos de entrenamiento estableciendo para la función 'fit' el número de iteraciones ('epochs'), el porcentaje de datos de validación ('validation_split') y el tamaño de lote para la retropropagación ('batch_size').
- **OPCIONAL**: En lugar de cablear en el código los parámetros anteriores y modificarlos a mano, se valorará muy positivamente la escritura de un programa que genere dinámicamente diferentes arquitecturas probando distintos valores para todos o algunos de los parámetros del punto anterior.

In [ ]:
# Código con la solución de la tarea #
